In [1]:
import random
import sys
import time

from pyspark import SparkConf
from pyspark import SparkContext
conf = SparkConf().setMaster("local").setAppName("OneMaxPython")
sc = SparkContext(conf = conf)

In [2]:
# Functions Parts

def MakePop(pop_size, ind_size): 
    # Create Populations for initialization
    lisPop = []
    for i in range(pop_size):
        lisPop.append([[random.randint(0,1) for i in range(ind_size)], 0])
    return lisPop

def EvaForEachInd(ele):
    # Fitness
    ele[1] = sum(ele[0])/len(ele[0])
    return ele

In [3]:
#选择精英者和剩下的群体
def Select(fitRDD, POPULATION_SIZE, ELITE_SIZE):
# 对评价过的种群按适应度从大到小进行排序
    sortedPopRDD = fitRDD.sortBy((lambda x: x[1]), False)
# 取出精英并创建精英RDD
    lisElite = sortedPopRDD.take(ELITE_SIZE)
    eliteRDD = sc.parallelize(lisElite)
# 取出剩下的种群
    RemainPop = fitRDD.sortBy(lambda x: x[1]).take(POPULATION_SIZE - ELITE_SIZE)
    random.shuffle(RemainPop)
    RemainPopRDD = sc.parallelize(RemainPop)
    return eliteRDD, RemainPopRDD

In [56]:
# 交叉
def crossover(a, b): # 两个个体交叉操作
    # Two-point crossover
    a_chr = a # 取出a的染色体
    b_chr = b # 取出b的染色体
    
    size = len(a_chr) # 取出染色体的长度

    f = random.randint(0, size) # 选取两个基因点，准备交叉
    s = random.randint(f, size)

    _a = a_chr[:f] + b_chr[f:s] + a_chr[s:]
    _b = b_chr[:f] + a_chr[f:s] + b_chr[s:]
    
    return _a, _b

def CROSSOVER(ele): # RDD层面的交叉操作
    
    a_Chromo, b_Chromo = crossover(ele[0][0],ele[1][0])
    ele[0][0] = a_Chromo
    ele[0][1] = 0
    ele[1][0] = b_Chromo
    ele[1][1] = 0
    return ele[0],ele[1]

In [5]:
# 变异部分
def MutationForInd(gene): # 基因变异
    global GENE_MUTATION
    for i in gene:
        if GENE_MUTATION > (random.randint(0, 100) / 100):
            i = random.randint(0,1)
    return gene
            
def Mutation(ele):  # 选择变异的个体，RDD 层面
    global INDIVIDUAL_MUTATION
    if INDIVIDUAL_MUTATION > (random.randint(0, 100) / 100):
        ele[0] = MutationForInd(ele[0])
    return ele

In [6]:
random.seed(64) # 随机种子设置
#---------------------------------------
# Constant Variables
CHROMOSOME_SIZE = 10 # 染色体尺寸
GENE_MUTATION = 0.05 # 基因变异率
INDIVIDUAL_MUTATION = 0.2 # 个体变异率
CROSSOVER = 0.5
POPULATION_SIZE = 40 # 种群数量
ELITE_PERCENTAGE = 0.5
#ELITE_SIZE = int(POPULATION_SIZE * ELITE_PERCENTAGE)
ELITE_SIZE = 2 
GENERATION_MAX = 100 # 最大迭代次数
#------------------------------------------------
# starts
start = time.clock() # 开始计时

In [7]:
population = MakePop(POPULATION_SIZE, CHROMOSOME_SIZE) # initial population

In [8]:
popRDD = sc.parallelize(population) 
fitRDD = popRDD.map(EvaForEachInd)
fitValues = [ele[1] for ele in fitRDD.collect()]


In [9]:
# Fitness statistics 
print("{0} Generation ---".format(1))
print("\tMIN: {0}".format(min(fitValues)))
print("\tMAX: {0}".format(max(fitValues)))
print("\tAVG: {0}".format(round(sum(fitValues) / len(fitValues), 3)), "\n")


1 Generation ---
	MIN: 0.2
	MAX: 0.8
	AVG: 0.495 



In [10]:
# select elites and remained populations
eliteRDD, RemainPopRDD = Select(fitRDD, POPULATION_SIZE, ELITE_SIZE)


In [11]:
RemainPopList = RemainPopRDD.collect()

In [22]:
INT = int(len(RemainPopList)/2)

In [59]:
PairRDD = sc.parallelize(RemainPopList, len(RemainPopList)/2).glom()

In [60]:
ELE = PairRDD.take(1)
ELE

[[[[1, 0, 1, 1, 1, 1, 1, 0, 0, 1], 0.7],
  [[1, 1, 1, 1, 0, 1, 1, 1, 0, 0], 0.7]]]

In [61]:
for i in ELE:
    print(i)
    print("--------")
print(ELE[0])
print(ELE[0][0])
print(ELE[0][1])
print(ELE[0][0][0])
print(ELE[0][0][1])
print(ELE[0][1][0])
print(ELE[0][1][1])

[[[1, 0, 1, 1, 1, 1, 1, 0, 0, 1], 0.7], [[1, 1, 1, 1, 0, 1, 1, 1, 0, 0], 0.7]]
--------
[[[1, 0, 1, 1, 1, 1, 1, 0, 0, 1], 0.7], [[1, 1, 1, 1, 0, 1, 1, 1, 0, 0], 0.7]]
[[1, 0, 1, 1, 1, 1, 1, 0, 0, 1], 0.7]
[[1, 1, 1, 1, 0, 1, 1, 1, 0, 0], 0.7]
[1, 0, 1, 1, 1, 1, 1, 0, 0, 1]
0.7
[1, 1, 1, 1, 0, 1, 1, 1, 0, 0]
0.7


In [62]:
CrossedRDD = PairRDD.flatMap(CROSSOVER)

In [63]:
CrossedRDD.collect()

[[[1, 0, 1, 1, 0, 1, 1, 0, 0, 1], 0],
 [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0], 0],
 [[0, 1, 0, 0, 0, 0, 1, 1, 1, 1], 0],
 [[0, 1, 1, 0, 1, 0, 0, 1, 1, 0], 0],
 [[0, 1, 1, 0, 0, 0, 0, 0, 1, 0], 0],
 [[1, 0, 0, 1, 0, 1, 1, 0, 0, 1], 0],
 [[0, 0, 1, 1, 0, 0, 0, 0, 1, 1], 0],
 [[1, 1, 1, 1, 0, 0, 1, 1, 1, 0], 0],
 [[0, 1, 1, 0, 0, 0, 1, 0, 1, 0], 0],
 [[1, 1, 0, 0, 1, 0, 1, 0, 1, 0], 0],
 [[0, 1, 0, 0, 0, 1, 0, 1, 0, 0], 0],
 [[0, 0, 1, 0, 1, 0, 1, 0, 0, 0], 0],
 [[0, 1, 1, 0, 0, 1, 1, 0, 0, 0], 0],
 [[1, 0, 0, 0, 0, 0, 1, 0, 1, 0], 0],
 [[0, 0, 1, 0, 1, 0, 1, 0, 0, 1], 0],
 [[0, 0, 0, 1, 1, 1, 1, 0, 1, 1], 0],
 [[0, 1, 0, 1, 1, 1, 1, 1, 0, 1], 0],
 [[1, 0, 1, 1, 0, 1, 0, 0, 0, 0], 0],
 [[0, 1, 1, 1, 1, 0, 0, 0, 1, 1], 0],
 [[1, 1, 0, 0, 0, 1, 1, 1, 1, 0], 0],
 [[0, 0, 1, 0, 0, 1, 1, 0, 0, 1], 0],
 [[1, 0, 1, 0, 0, 1, 0, 0, 1, 1], 0],
 [[1, 0, 1, 1, 1, 0, 0, 0, 0, 1], 0],
 [[0, 1, 1, 1, 1, 1, 0, 1, 1, 1], 0],
 [[0, 0, 0, 0, 1, 0, 0, 0, 1, 0], 0],
 [[0, 0, 1, 0, 0, 1, 0, 0, 1, 0], 0],
 [[0, 0, 0, 